In [35]:
from bs4 import BeautifulSoup
import requests
from pprint import pprint
import pandas as pd

In [36]:
# 変数urlにSUUMOホームページのURLを格納する。東京都／目黒区 15万円以上～20万円以下 2LDK／3LDKのURLを格納
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13110&ta=13&cb=15.0&ct=20.0&md=07&md=10&et=9999999&mb=0&mt=9999999&cn=9999999&fw2=&?page={}'

# アクセスするためのURLをtarget_urlに格納する
target_url = url.format(1)

# print()してtarget_urlを確認する
print(target_url)

https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13110&ta=13&cb=15.0&ct=20.0&md=07&md=10&et=9999999&mb=0&mt=9999999&cn=9999999&fw2=&?page=1


In [37]:
# target_urlへのアクセス結果を、変数rに格納
r = requests.get(target_url)

# 取得結果を解析してsoupに格納
soup = BeautifulSoup(r.text)

In [62]:
# 変数d_listに空のリストを作成する
d_list = []

# すべての物件情報(20件)を取得する
contents = soup.find_all('div', class_='cassetteitem')

# 各物件情報をforループで取得する
for content in contents:
    # 物件情報と部屋情報を取得しておく
    detail = content.find('div', class_='cassetteitem_content')
    table = content.find('table', class_='cassetteitem_other')
    
    # 物件情報から必要な情報を取得する
    title = detail.find('div', class_='cassetteitem_content-title').text
    address = detail.find('li', class_='cassetteitem_detail-col1').text
    access = detail.find('li', class_='cassetteitem_detail-col2').text
    age = detail.find('li', class_='cassetteitem_detail-col3').text

    # 部屋情報のブロックから、各部屋情報を取得する
    tr_tags = table.find_all('tr', class_='js-cassette_link')
    
    # 各部屋情報をforループで取得する
    for tr_tag in tr_tags:        
        
        # 部屋情報の行から、欲しい情報を取得する
        floor, price, first_fee, capacity = tr_tag.find_all('td')[2:6]
        
        # さらに細かい情報を取得する
        fee, management_fee = price.find_all('li')
        deposit, gratuity = first_fee.find_all('li')
        madori, menseki = capacity.find_all('li')

        # 取得したすべての情報を辞書に格納する
        d = {
            '物件名': title,
            '住所': address,
            'アクセス': access,
            '築年数': age,
            '階数': floor.text,
            '賃料': fee.text,
            '管理費': management_fee.text,
            '敷金': deposit.text,
            '礼金': gratuity.text,
            '間取り': madori.text,
            '面積': menseki.text
        }
        
        # 取得した辞書をd_listに格納する
        d_list.append(d)

In [63]:
df = pd.DataFrame(d_list)

In [64]:
df

,物件名,住所,アクセス,築年数,階数,賃料,管理費,敷金,礼金,間取り,面積
0,菅野ビル,東京都目黒区中根１,\n東急東横線/都立大学駅 歩3分\n東急大井町線/自由が丘駅 歩16分\n東急目黒線/奥沢...,\n築34年\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t4階,19.5万円,5000円,19.5万円,-,2LDK,74.77m2
1,菅野ビル,東京都目黒区中根１,\n東急東横線/都立大学駅 歩3分\n東急大井町線/自由が丘駅 歩16分\n東急目黒線/奥沢...,\n築34年\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t4階,19.5万円,5000円,19.5万円,-,2LDK,74.77m2
2,菅野ビル,東京都目黒区中根１,\n東急東横線/都立大学駅 歩3分\n東急大井町線/自由が丘駅 歩16分\n東急目黒線/奥沢...,\n築34年\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t4階,19.5万円,5000円,19.5万円,-,2LDK,74.77m2
3,ストークハイツ中島,東京都目黒区上目黒３,\n東急東横線/中目黒駅 歩12分\n東急東横線/祐天寺駅 歩10分\n東急田園都市線/池尻...,\n築33年\n2階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,19万円,-,19万円,19万円,3LDK,60.3m2
4,東急東横線 学芸大学駅 8階建 築51年,東京都目黒区中央町２,\n東急東横線/学芸大学駅 歩10分\n東急東横線/祐天寺駅 歩12分\n東急東横線/中目黒...,\n築51年\n8階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t4階,15.5万円,6000円,15.5万円,15.5万円,3LDK,49.5m2
5,Ｇｒｅｅｎ Ｆｏｒｅｓｔ Ｇｏｈｏｎｇｉ,東京都目黒区五本木３,\n東急東横線/学芸大学駅 歩7分\n東急東横線/祐天寺駅 歩13分\n東急田園都市線/三軒...,\n築2年\n2階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,18万円,3000円,-,18万円,2LDK,58.03m2
6,ネスト鷹番,東京都目黒区鷹番１,\n東急東横線/学芸大学駅 歩8分\n東急東横線/都立大学駅 歩18分\nＪＲ山手線/目黒駅...,\n築22年\n3階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,20万円,5000円,20万円,20万円,2LDK,55.2m2
7,東急田園都市線 池尻大橋駅 3階建 築18年,東京都目黒区青葉台３,\n東急田園都市線/池尻大橋駅 歩10分\n京王井の頭線/神泉駅 歩12分\n東急東横線/中...,\n築18年\n3階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t2階,18.4万円,4000円,18.4万円,27.6万円,2LDK,56.74m2
8,コートガーディニア,東京都目黒区祐天寺１,\n東急東横線/祐天寺駅 歩5分\n東急東横線/中目黒駅 歩11分\n東京メトロ日比谷線/中...,\n築28年\n4階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t3階,20万円,5000円,20万円,20万円,2LDK,47.94m2
9,東急東横線 都立大学駅 5階建 築34年,東京都目黒区中根１,\n東急東横線/都立大学駅 歩2分\n\n\n,\n築34年\n5階建\n,\r\n\t\t\t\t\t\t\t\t\t\t\t4階,19.5万円,5000円,19.5万円,-,2LDK,74.77m2


In [65]:
# dfをcsvに出力
df.to_csv('suumo.csv', index=None, encoding='utf-8-sig')